In [9]:
%pip install minio
%pip install dbfread
%pip install pyspark

In [10]:

from pyspark.sql import SparkSession
import os
from dbfread import DBF
from minio import Minio
import logging
import pandas as pd

In [3]:
'''
spark = SparkSession.builder.master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,"
                                    "org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.hadoop.fs.s3a.access.key", "cM0ahX6pSrZAJ34fO7mM") \
    .config("spark.hadoop.fs.s3a.secret.key", "FewZNI1GV22FH5LOmhMY6irrJm9UNF91pjX8bfBS") \
    .config("spark.hadoop.fs.s3a.endpoint", "10.100.100.61:9000") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

client = Minio(
    "10.100.100.61:9000",
    access_key="EsVj1dnneqMciIYcBbhR",
    secret_key="oolW2uY06jXIoE2oMNeE8MSgskzuz14HaFYdhEh7",
    secure=False
)

'''

client = Minio(
    "localhost:9000", 
    secure=False,
    access_key="wiqq5Pbe7l6rT2Pm4GGg",
    secret_key="wY9ChEaK7BOFoniIo2rTwfE6ZYK6bE0eCJuJonX6",
)

spark = SparkSession.builder.appName("teste").getOrCreate()

In [13]:
if __name__ == '__main__':
    logger = logging.getLogger(__name__)
    logging.basicConfig(encoding='utf-8', level=logging.INFO)
    base_path = os.getcwd()

    filename = "DOSP2020.dbf"
    path_sinasc_2021 = base_path + f"./dados/sinasc_data_raw_2021/{filename}"
    bucket_name = "sinasc"

    already_bucket = client.bucket_exists(bucket_name)

    if not already_bucket:
        client.make_bucket(bucket_name)
        logger.info("Created bucket " + bucket_name)
    else:
        logger.info("Bucket " + bucket_name + " already exists")

    remote_path = f"s3a://{bucket_name}/{filename}"
    records_sinasc_2021 = DBF(path_sinasc_2021)
 
    df_sinasc_2021 = spark.createDataFrame(records_sinasc_2021)\
        .write\
        .format("delta")\
        .mode("overwrite")\
        .option("compression", "snappy")\
        .save(remote_path)
    
    logger.info(f"Arquivo {filename} salvo em {remote_path} com sucesso.")

INFO:__main__:Bucket sinasc already exists


KeyboardInterrupt: 